In [4]:
from gerrychain import Graph, Election, updaters, Partition, MarkovChain, constraints
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from gerrychain.tree import random_spanning_tree, PopulatedGraph, predecessors
from collections import deque, namedtuple
from functools import partial
from collections import Counter
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
Cut = namedtuple("Cut", "edge subset")


##########################################################################################
#     THIS IS NO LONGER NEEDED, BUT IS HANDY FOR CHECKING THE SIZE OF THE PARTITIONS     #
#                 TURN IT OFF AND REMOVE CONSTRAINT FOR ACTUAL RUNS                      #
##########################################################################################
# def stop_disappearing(partition):                                                      #
#     if not partition.parent:                                                           #
#         return True                                                                    #
#     sizes_now = [len(part) for part in partition.assignment.parts.values()]            #
#     sizes_past = [len(part) for part in partition.parent.assignment.parts.values()]    #
#     print(sizes_now, min(sizes_past), max(sizes_past))                                 #
#     return all(len(part) > 1 for part in partition.assignment.parts.values())          #
##########################################################################################


def find_balanced_edge_cuts(h, choice=random.choice):
    root = choice([x for x in h if h.degree(x) > 2])
    pred = predecessors(h.graph, root)
    cuts = []
    leaves = deque(x for x in h if h.degree(x) == 1)
    while len(leaves) > 0:
        leaf = leaves.popleft()
        cuts.append(Cut(edge=(leaf, pred[leaf]), subset=h.subsets[leaf].copy()))
        parent = pred[leaf]
        h.contract_node(leaf, parent)
        if h.degree(parent) == 1 and parent != root:
            leaves.append(parent)
            
    even_split = int(len(h.graph.nodes)/2)
    #cut_lengths is number of nodes in one of the districts after cut of a specific edge
    cut_lengths = np.asarray([len(cuts[i].subset) for i in range(len(cuts))])
    cut_distances = np.abs(cut_lengths-even_split)
    cut_distances, cuts = zip(*sorted(zip(cut_distances, cuts)))
    cut_distances = list(filter(lambda a: a != max(cut_distances), cut_distances))
    
    temp_d = {}
    for i in range(len(cut_distances)):
        temp_d[i] = cut_distances[i]
    #print(temp_d)
        # indices of cut_distances
    population = list(range(len(cut_distances)))
    weight_small_d = (1/(np.asarray(cut_distances)**2+1))
    probs = weight_small_d/sum(weight_small_d)
#     print(probs)
    choice = random.choices(population, probs)[0]
#     print(choice)
############################################################################## 
#   VERIFIES THAT YOU ARE SAMPLING FROM ALL POSSIBLE EDGE CHOICES CORRECTLY  #
##############################################################################
#     d = Counter(random.choices(population, probs, k=100000))               #
#     fig = plt.figure(figsize=(12,12))                                      #
#     ax = fig.add_subplot(2, 1, 1)                                          #
#     plt.bar(d.keys(), d.values(), width=1.0, color='g')                    #
#     ax.set_yscale('log')                                                   #
#     plt.show()                                                             #
#     print(d)                                                               #
##############################################################################
    return [cuts[choice]]


def partition_tree(graph, pop_col, pop_target, epsilon, node_repeats=1, spanning_tree=None, choice=random.choice):
    populations = {node: graph.nodes[node][pop_col] for node in graph}
    possible_cuts = []
    spanning_tree = random_spanning_tree(graph)
    while len(possible_cuts) == 0:
        spanning_tree = random_spanning_tree(graph)
        h = PopulatedGraph(spanning_tree, populations, pop_target, epsilon)
        possible_cuts = find_balanced_edge_cuts(h, choice=choice)
    return choice(possible_cuts).subset

In [5]:
###################################################################
#                    *HASHING AND COMPRESSING*                    #
# THE PythonObjectEncoder CLASS SERIALIZES GENERIC PYTHON OBJECTS #
#         THESE OBJECTS CAN THEN BE COMPRESSED AND SAVED          #
###################################################################
from json import JSONEncoder, dumps, loads
from io import BytesIO
import hashlib
import pickle
import gzip
from os import path


class PythonObjectEncoder(JSONEncoder):
    def __init__(self, partition):
        self.partition = partition
        self.keys = sorted(list(initial_partition.parts.keys()))
        self.objstrings = []
        self.hashstrings = []
        self.d_plan = dict.fromkeys(self.keys)
        
        
    def __call__(self, obj):
        if isinstance(obj, (list, dict, str, bytes, int, float, bool, type(None))):
            return JSONEncoder.default(self, obj)
        return {'_python_object': pickle.dumps(obj)}
    
    
    def update(self, partition):
        for i in range(len(self.keys)):
            self.d_plan[self.keys[i]] = list(partition.assignment.parts[self.keys[i]])
        encoded_plan = dumps(self.d_plan)
        compressed_plan = PythonObjectEncoder.compressStringToBytes(encoded_plan)
        hashstring = hashlib.md5(encoded_plan.encode('utf-8')).hexdigest()
        self.objstrings.append(compressed_plan)
        self.hashstrings.append(hashstring)
        
        
    def commit(self, file_name):
        final_table = dict(zip(self.hashstrings,self.objstrings))
        self.save_obj(final_table, file_name)
        
        
    def load_maps(self, file_name):
        decode_test = self.load_obj(file_name)
        recreated_maps = []
        for key in decode_test:
            recreated_maps.append(loads(PythonObjectEncoder.decompressBytesToString(decode_test[key])))
        return recreated_maps
        
    
    @staticmethod
    def compressStringToBytes(inputString):
        bio = BytesIO()
        bio.write(inputString.encode("utf-8"))
        bio.seek(0)
        stream = BytesIO()
        compressor = gzip.GzipFile(fileobj=stream, mode='w')
        while True:
            chunk = bio.read(8192)
            if not chunk:
                compressor.close()
                return stream.getvalue()
            compressor.write(chunk)

            
    @staticmethod
    def decompressBytesToString(inputBytes):
        bio = BytesIO()
        stream = BytesIO(inputBytes)
        decompressor = gzip.GzipFile(fileobj=stream, mode='r')
        while True:
            chunk = decompressor.read(8192)
            if not chunk:
                decompressor.close()
                bio.seek(0)
                return bio.read().decode("utf-8")
            bio.write(chunk)
        return None
    
    
    @staticmethod
    def save_obj(obj, name):
        if path.exists(name + '.pkl'):
            prev_map = PythonObjectEncoder.load_obj(name)
            final_map = {**prev_map, **obj}
            with open(name + '.pkl', 'rb+') as f:
                pickle.dump(final_map, f, pickle.HIGHEST_PROTOCOL)
        else:
            with open(name + '.pkl', 'wb') as f:
                pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
        
    @staticmethod
    def load_obj(name):
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)

In [ ]:
######################################################
##############     importing libraries  ##############
######################################################

from gerrychain import (
                        Graph, Partition, Election,
                        updaters, constraints, accept,
                        MarkovChain
                        )
from gerrychain.updaters import Tally, cut_edges, perimeter, exterior_boundaries, boundary_nodes, interior_boundaries, cut_edges_by_part
from gerrychain.metrics import polsby_popper, efficiency_gap
from gerrychain.proposals import recom, propose_random_flip
from functools import partial
from gerrychain.accept import always_accept
import pandas
import random
import numpy as np
import tqdm
import sys


  #### Custom functions that allow us to create conditions to accept a move
filename = "TX_vtds_fixed"
graph = Graph.from_json(f"{filename}.json")

# Creates a way to quickly change election office, election year, assignment map, and population
office, year = "PRES", "16"
pop_used = "TOTPOP"
area = "area"
assign = "USCD"

steps = 2000
epsil = 9999

# Finishes creation of quick refereces for election cycles
elect = office + year
elect_D = elect + "D"
elect_R = elect + "R"

elections = Election(elect, {"Dem": elect_D, "Rep": elect_R})

k_in = 1
    
def objective_score(partition):
    comp = sum(polsby_popper(partition).values())/(len(partition))
    return comp


def custom_objective(partition):
    if partition.parent == None:
        return False
    else:
        new = objective_score(partition)
        old = objective_score(partition.parent)
        if new > old:
            return True
        else:
            bound = min(1, new/old)
            rand = random.random()
            return rand <= bound

# def noWorseThanInitial(partition):
#     if sum(polsby_popper(partition).values())/len(partition) >= sum(polsby_popper(initial_partition).values())/len(initial_partition):
#         return True
#     else:
#         return False


######################################################
################     creating graph  #################
######################################################
i=0

while i < 100:


    #######################################################
    ###########     creating initial partition  ###########
    #######################################################

    initial_partition = Partition(
        graph,
        assignment=assign,
        updaters={
            #"cut_edges": cut_edges,
            "population": Tally(pop_used, alias="population"),
            #"area": Tally(area, alias="area"),
            #"exterior_boundaries": exterior_boundaries,
            #"interior_boundaries": interior_boundaries,
            #"boundary_nodes": boundary_nodes,
            #"cut_edges_by_part": cut_edges_by_part,
            #"perimeter": perimeter,
            elect: elections
        }
    )



    ######################################################
    ###################     proposal     #################
    ######################################################

    ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

    proposal = partial(recom,
                       pop_col=pop_used,
                       pop_target=ideal_population,
                       epsilon=epsil,
                       node_repeats=2,
                       method=partition_tree
                      )

    ######################################################
    ##############     compactness bound  ###############
    ######################################################

    # Compactness Bound Constraint
    # compactness_bound = constraints.UpperBound(
    #     lambda p: len(p["cut_edges"]),
    #     2*len(initial_partition["cut_edges"])
    # )

    # Must be 0.05 for SEND runs
    # pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, epsil)


    ######################################################
    #################     markov chain  ##################
    ######################################################

    chain = MarkovChain(
        proposal=proposal,        
        accept=always_accept,
        constraints = [], #pop_constraint
        initial_state=initial_partition,
        total_steps=steps
    )

    # Run the chain, putting the sorted Democratic vote percentages
    # into a pandas DataFrame for analysis and plotting.

    # This will take about 10 minutes.
    
    poe = PythonObjectEncoder(initial_partition)
    dem_data = []
    k=0
    for partition in chain:
        dem_data.append(sorted(partition[elect].percents("Dem")))
        poe.update(partition)
        poe.commit('test2')
        print(k)
        k+=1
    data = pd.DataFrame(dem_data)
    
    #print(data)

    print('second check')
    for k in np.arange(0,2000,1):
        data2 = pandas.DataFrame(data.iloc[k]).transpose().reset_index(drop=True)
        data2['StepNum'] = k+1
        data2['RunNum'] = i

        if (i == 0) and (k == 0):
            with open('Week_Run_no_con_wo_pols_e_9999{0}.csv'.format(int(epsil*100)), 'a') as file:
                data2.to_csv(file, header=True)
        else:
            with open('Week_Run_no_con_wo_pols_e_9999{0}.csv'.format(int(epsil*100)), 'a') as file:
                data2.to_csv(file, header=False)

    print("Previously Completed Step: {0}".format(i))
    
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27